In [4]:
import pygame
import random


sound1 = 'ding.mp3'
imgelv = "elv.png"
imgefloor = "b.png"
imgpassenger = "men1.png"
class Building:
    num_floors = 8

    def __init__(self, screen, floorheight, floor_width, num_of_elv=2, space=1):
        self.screen = screen
        self.elevators = [Elevator(screen, 250 + j *300, floorheight) for j in range(num_of_elv)]
        self.floors = [Floor(screen, 0, i * floorheight, floorheight, floor_width, i, self) for i in range(Building.num_floors)]

    def update(self, floor, direction):
        selectelevator = self.elevators[0]
        min_distance = abs(selectelevator.myfloor - floor.floor_num)
        for elevator in self.elevators:
            if elevator.is_moving_in_direction(floor.floor_num, direction):
                distance = abs(elevator.myfloor - floor.floor_num)
                if distance < min_distance:
                    min_distance = distance
                    selectelevator = elevator
        selectelevator.add_target(floor)
        selectelevator.move()

    def draw(self):
        for floor in self.floors:
            floor.draw()
        for elevator in self.elevators:
            elevator.draw()
    
        
            

class Elevator:
    def __init__(self, screen, initial_posx, floorheight, step=10):
        self.screen = screen
        self.y_pos = 0
        self.x_pos = initial_posx
        self.floorheight = floorheight
        self.step = floorheight/100  
        self.myfloor = self.y_pos // self.floorheight
        self.direction = None
        self.targets = []
        self.passengers =[]
        pygame.mixer.init()
        self.ding_sound = pygame.mixer.Sound(sound1)
        
        original_image = pygame.image.load(imgelv).convert_alpha()
        self.image = pygame.transform.scale(original_image, (100, floorheight))
        self.image = pygame.transform.scale(original_image, (75, int(floorheight * 0.75)))  
        self.rect = self.image.get_rect(topleft=(self.x_pos, self.y_pos))

    def add_target(self, floor):
        if floor not in self.targets:
            self.targets.append(floor)
            self.targets.sort(key=lambda x: x.floor_num, reverse=self.direction == -1)
        if self.direction is None:
            self.set_direction(floor.floor_num)

    def set_direction(self, floor_num):
        if self.myfloor < floor_num:
            self.direction = 1
        elif self.myfloor > floor_num:
            self.direction = -1

    def move(self):
       a=0
       while a==0:
       
        self.draw()
        if not self.targets:
            return
        if self.direction is None:
            print("Direction not set, cannot move.")
            return
        self.rect.y += self.direction
        
        self.myfloor = self.rect.y // self.floorheight
        target_floor = self.targets[0]
        if self.myfloor == target_floor.floor_num:
            self.ding_sound.play()
            a=1
            target_floor.load_passengers(self,target_floor)
            self.targets.pop(0)
            if not self.targets:
                self.direction = None
        pygame.time.set_timer(pygame.USEREVENT, 100)

    def is_moving_in_direction(self, floor_num, direction):
        if self.direction is None:
            return True
        return (self.direction == 1 and self.myfloor <= floor_num) or (self.direction == -1 and self.myfloor >= floor_num)

    def draw(self):
        self.screen.blit(self.image, self.rect)

        font = pygame.font.Font(None, 36)  

        text_surface = font.render(str(self.rect.y), True, (0, 0, 0))  
        text_rect = text_surface.get_rect(center=(self.rect.centerx, self.rect.y + 20))  

        self.screen.blit(text_surface, text_rect)

        button_text = font.render(str(self.myfloor), True, (255, 255, 255))  
        button_rect_width = button_text.get_width() + 10  
        button_rect_height = button_text.get_height() + 10
        button_rect = pygame.Rect(self.rect.x + 10, self.rect.bottom - button_rect_height - 10, button_rect_width, button_rect_height)  

        pygame.draw.rect(self.screen, (0, 128, 0), button_rect)  

        button_text_rect = button_text.get_rect(center=button_rect.center)
        self.screen.blit(button_text, button_text_rect)
        for passenger in self.passengers:
            self.screen.blit(passenger.image, (self.rect.x + 10, self.rect.y + 20 * self.passengers.index(passenger)))

            

class Floor:
    def __init__(self, screen, pos_x, pos_y, height, width, floor_num, building):
        self.screen = screen
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.width = width
        self.height = height
        self.floor_num = floor_num
        self.building = building
        self.passengers = []
        
        brick_image = pygame.image.load(imgefloor).convert_alpha()
        self.brick_texture = pygame.transform.scale(brick_image, (width, height))
        self.rect = self.brick_texture.get_rect(topleft=(pos_x, pos_y))
        self.button_rect = pygame.Rect(pos_x + width - 100, pos_y + 10, 90, 30)

        self.passengers.append(Passenger(self.floor_num))
    def draw(self):
        self.screen.blit(self.brick_texture, self.rect)
        font = pygame.font.Font(None, 36)
        text_surface = font.render(str(self.floor_num), True, (0, 0, 0))
        text_rect = text_surface.get_rect(center=(self.pos_x + self.width // 2, self.pos_y + self.height // 2))
        self.screen.blit(text_surface, text_rect)
        pygame.draw.rect(self.screen, (0, 128, 0), self.button_rect)
        button_text = font.render(str(self.floor_num), True, (255, 255, 255))
        button_text_rect = button_text.get_rect(center=self.button_rect.center)
        self.screen.blit(button_text, button_text_rect)
        for passenger in self.passengers:
            self.screen.blit(passenger.image, (self.rect.x + 10, self.rect.y + 20 * self.passengers.index(passenger)))

    def check_click(self, position):
        if self.button_rect.collidepoint(position):
            self.call_elevator()

    def call_elevator(self):
        direction = 1 if any(p.destination > self.floor_num for p in self.passengers) else -1
        self.building.update(self, direction)

    def load_passengers(self, elevator,floor):
        to_remove = []
        for passenger in self.passengers:
            if (passenger.destination  ==self.floor_num ):
                elevator.passengers.append(passenger)
                to_remove.append(passenger)
                print(8)
        for passenger in floor.passengers:##להוסיף תנאי ליעד 
            # if (passenger.destination  ==self.floor_num ):
                elevator.passengers.append(passenger)
                to_remove.append(passenger)
                print(8) 
        for passenger in to_remove:
            floor.passengers.remove(passenger)

class Passenger:
    def __init__(self, current_floor):
        self.current_floor = current_floor
        self.destination = self.pick_random_destination()
        self.image = pygame.image.load(imgpassenger).convert_alpha()  


    def pick_random_destination(self):
        
        potential_destinations = list(range(Building.num_floors))
        potential_destinations.remove(self.current_floor)
        return random.choice(potential_destinations)

def run_game():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption('Building Simulation')
    clock = pygame.time.Clock()
    building = Building(screen, 70, 200) 

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                print("Mouse Clicked at:", event.pos) 
                if event.button == 1:  
                    for floor in building.floors:
                        floor.check_click(event.pos)

        screen.fill((255, 255, 255))  
        building.draw()  
        pygame.display.flip()  
        clock.tick(60)  

    pygame.quit()

run_game()


Mouse Clicked at: (141, 229)
8
Mouse Clicked at: (165, 377)
8
Mouse Clicked at: (163, 443)
8
Mouse Clicked at: (143, 169)
8
Mouse Clicked at: (156, 89)
8
Mouse Clicked at: (129, 307)
8
